In [7]:
import os
import cv2
import yaml
import random
import numpy as np

# 設定原始資料夾
dataset_path = "Convert-Dataset-New-2"
output_path = "dataset-yolo-cls-cropped"

# 設定最小影像尺寸（例如 32x32）
min_size = 32
background_samples = 2  # 每張圖產生 5 個 background 樣本

# 讀取 data.yaml 獲取類別名稱
yaml_path = os.path.join(dataset_path, "data.yaml")
with open(yaml_path, "r", encoding="utf-8") as f:
    data_config = yaml.safe_load(f)

# 取得類別名稱對應
class_names = data_config["names"]  # 這是一個 list，對應類別 ID

# **合併 front_* 和 back_* 的類別**
class_mapping = {
    "front_wheel": "wheel", "back_wheel": "wheel",
    "front_pedal": "pedal", "back_pedal": "pedal",
    "front_handle": "handle", "back_handle": "handle",
    "front_mudguard": "mudguard", "back_mudguard": "mudguard",
    "front_handbreak": "handbrake", "back_hand_break": "handbrake",
    "front_light": "light", "back_light": "light",
    "front_reflector": "reflector", "back_reflector": "reflector",
    "saddle": "saddle",
    "bell": "bell", "chain": "chain", "kickstand": "kickstand",
    "lock": "lock", "steer": "steer", "gear_case": "gear_case",
    "dress_guard": "dress_guard", "dynamo": "dynamo"
}

# 取得合併後的唯一類別列表
final_classes = ["background"] + sorted(set(class_mapping.values()))  # 背景=0，其餘類別依序排序

# 建立新資料夾
os.makedirs(output_path, exist_ok=True)

# 處理 Train 和 Val
for split in ["train", "valid"]:
    images_dir = os.path.join(dataset_path, split, "images")
    labels_dir = os.path.join(dataset_path, split, "labels")

    if not os.path.exists(labels_dir):
        continue  # 如果沒有標註，跳過

    # 讀取標註檔案
    for label_file in os.listdir(labels_dir):
        if not label_file.endswith(".txt"):
            continue

        label_path = os.path.join(labels_dir, label_file)
        image_name = label_file.replace(".txt", ".jpg")
        image_path = os.path.join(images_dir, image_name)

        if not os.path.exists(image_path):
            continue  # 確保影像存在

        # 讀取影像
        image = cv2.imread(image_path)
        if image is None:
            continue

        img_height, img_width, _ = image.shape  # 取得影像尺寸

        # 讀取標註檔案
        with open(label_path, "r") as f:
            lines = f.readlines()

        # 初始化標註類別
        detected_classes = set()
        bounding_boxes = []

        # **處理每一個 bounding box**
        for idx, line in enumerate(lines):
            parts = line.strip().split()
            if len(parts) < 5:
                continue  # 確保資料完整

            class_id = int(parts[0])  # 類別 ID
            class_name = class_names[class_id]  # 取得對應的類別名稱

            # **判斷類別是否在合併對應表**
            if class_name in class_mapping:
                merged_class_name = class_mapping[class_name]  # 轉換成合併後的類別名稱
                final_class_id = final_classes.index(merged_class_name)  # 取得新索引
            else:
                continue  # 忽略未定義的類別

            # 取得 YOLO 標註的 bounding box
            x_center, y_center, width, height = map(float, parts[1:])

            # 轉換相對座標為實際像素座標
            x1 = int((x_center - width / 2) * img_width)
            y1 = int((y_center - height / 2) * img_height)
            x2 = int((x_center + width / 2) * img_width)
            y2 = int((y_center + height / 2) * img_height)

            # 確保座標不超出範圍
            x1, y1 = max(0, x1), max(0, y1)
            x2, y2 = min(img_width, x2), min(img_height, y2)

            # **儲存 bounding box 座標**
            bounding_boxes.append((x1, y1, x2, y2))

            # 裁剪影像
            cropped_img = image[y1:y2, x1:x2]

            # **過濾掉過小的影像**
            if cropped_img.shape[0] < min_size or cropped_img.shape[1] < min_size:
                continue  # 忽略尺寸過小的影像

            # **建立資料夾**
            class_dir = os.path.join(output_path, split, merged_class_name)
            os.makedirs(class_dir, exist_ok=True)

            # 儲存裁剪影像
            cropped_filename = f"{image_name.replace('.jpg', '')}_{idx}.jpg"
            cropped_path = os.path.join(class_dir, cropped_filename)
            cv2.imwrite(cropped_path, cropped_img)

            detected_classes.add(final_class_id)  # 記錄此影像的類別

        # **隨機取 5 個「不與 bounding box 重疊」的區域作為背景**
        bg_dir = os.path.join(output_path, split, "background")
        os.makedirs(bg_dir, exist_ok=True)

        for i in range(background_samples):
            for _ in range(20):  # 最多嘗試 20 次以找到適合的背景區域
                rand_x = random.randint(0, img_width - min_size)
                rand_y = random.randint(0, img_height - min_size)
                rand_x2 = rand_x + min_size
                rand_y2 = rand_y + min_size

                # **檢查是否與 bounding box 重疊**
                overlap = False
                for (x1, y1, x2, y2) in bounding_boxes:
                    if not (rand_x2 < x1 or rand_x > x2 or rand_y2 < y1 or rand_y > y2):
                        overlap = True
                        break

                if not overlap:
                    bg_crop = image[rand_y:rand_y2, rand_x:rand_x2]
                    if bg_crop.shape[0] < min_size or bg_crop.shape[1] < min_size:
                        continue  # 確保背景區域不會太小
                    bg_filename = f"{image_name.replace('.jpg', '')}_bg_{i}.jpg"
                    bg_path = os.path.join(bg_dir, bg_filename)
                    cv2.imwrite(bg_path, bg_crop)
                    break  # 成功找到背景區域後跳出內部迴圈

# 處理完所有資料後，將 valid 重新命名為 val
valid_path = os.path.join(output_path, "valid")
val_path = os.path.join(output_path, "val")

if os.path.exists(valid_path):
    os.rename(valid_path, val_path)
    print(f"✅ `valid/` 已成功更名為 `val/`")
else:
    print(f"⚠️ `valid/` 目錄不存在，可能已經是 `val/` 或未生成。")

print("✅ YOLO-CLS 裁剪 & 類別合併完成！")


✅ YOLO-CLS 裁剪 & 類別合併完成！
✅ `valid/` 已成功更名為 `val/`
✅ YOLO-CLS 裁剪 & 類別合併完成！


In [8]:
# 設定資料集的根目錄
dataset_root = "dataset-yolo-cls-cropped"

# 設定 train/ 和 val/ 的路徑
train_dir = os.path.join(dataset_root, "train")
val_dir = os.path.join(dataset_root, "val")

# 檢查 train/ 是否存在
if not os.path.exists(train_dir):
    raise FileNotFoundError(f"❌ 錯誤: `{train_dir}` 不存在！請檢查你的資料集路徑。")

# 讀取類別名稱
class_names = sorted(os.listdir(train_dir))  # 取得類別名稱
num_classes = len(class_names)  # 計算類別數量

# 建立 YAML 內容
yaml_content = {
    "train": os.path.abspath(train_dir),  # 設定 train 資料夾的完整路徑
    "val": os.path.abspath(val_dir),      # 設定 val 資料夾的完整路徑
    "nc": num_classes,                    # 類別數量
    "names": class_names                   # 類別名稱
}

# 儲存 `data.yaml`
yaml_path = os.path.join(dataset_root, "data.yaml")
with open(yaml_path, "w", encoding="utf-8") as f:
    yaml.dump(yaml_content, f, default_flow_style=False, allow_unicode=True)

print(f"✅ `data.yaml` 已成功建立，路徑: {yaml_path}")

✅ `data.yaml` 已成功建立，路徑: dataset-yolo-cls-cropped/data.yaml


In [9]:
from ultralytics import YOLO

# 載入 YOLOv8 分類模型
model = YOLO("yolov8n-cls.pt")

model.train(data="./dataset-yolo-cls-cropped", epochs=20, imgsz=224, batch=64, device="cuda", amp=True)

New https://pypi.org/project/ultralytics/8.3.85 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.84 🚀 Python-3.9.21 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3090, 24161MiB)
engine/trainer: task=classify, mode=train, model=yolov8n-cls.pt, data=./dataset-yolo-cls-cropped, epochs=20, time=None, patience=100, batch=64, imgsz=224, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, sho

train: Scanning /home/user/LAB/Surveying_Camp/dataset-yolo-cls-cropped/train...


train: New cache created: /home/user/LAB/Surveying_Camp/dataset-yolo-cls-cropped/train.cache


val: Scanning /home/user/LAB/Surveying_Camp/dataset-yolo-cls-cropped/val... 159

val: New cache created: /home/user/LAB/Surveying_Camp/dataset-yolo-cls-cropped/val.cache


...

                   all      0.849      0.996



20 epochs completed in 0.430 hours.
Optimizer stripped from runs/classify/train4/weights/last.pt, 3.0MB
Optimizer stripped from runs/classify/train4/weights/best.pt, 3.0MB

Validating runs/classify/train4/weights/best.pt...
Ultralytics 8.3.84 🚀 Python-3.9.21 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3090, 24161MiB)
YOLOv8n-cls summary (fused): 30 layers, 1,456,657 parameters, 0 gradients, 3.3 GFLOPs
train: /home/user/LAB/Surveying_Camp/dataset-yolo-cls-cropped/train... found 111917 images in 17 classes ✅ 
val: /home/user/LAB/Surveying_Camp/dataset-yolo-cls-cropped/val... found 15943 images in 17 classes ✅ 
test: None...


               classes   top1_acc   top5_acc: 100%|██████████| 125/125 [00:07<0


                   all       0.85      0.996
Speed: 0.1ms preprocess, 0.1ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train4


ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fd5e928c670>
curves: []
curves_results: []
fitness: 0.9226306080818176
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.8496518731117249, 'metrics/accuracy_top5': 0.9956093430519104, 'fitness': 0.9226306080818176}
save_dir: PosixPath('runs/classify/train4')
speed: {'preprocess': 0.07245771599512794, 'inference': 0.0707121687559623, 'loss': 0.00015255955196807235, 'postprocess': 0.00027193006903600465}
task: 'classify'
top1: 0.8496518731117249
top5: 0.9956093430519104

In [13]:
import os
import yaml
import shutil  # 搬移檔案
import pandas as pd  # 儲存 CSV
from ultralytics import YOLO

# 載入訓練好的 YOLOv8 分類模型
final_model = YOLO("./runs/classify/train4/weights/last.pt")

# 設定輸入影像資料夾
input_folder = "./SL07_4"  # 你的影像資料夾

# 設定輸出結果資料夾 (依據 `input_folder` 命名)
output_root = f"{input_folder}_result"
saddle_output_folder = os.path.join(output_root, "saddle_output")  # ✅ 存放高機率 "saddle" 影像
not_saddle_output_folder = os.path.join(output_root, "not_saddle_output")  # ✅ 存放非 "saddle" 或低機率 saddle 影像
os.makedirs(saddle_output_folder, exist_ok=True)
os.makedirs(not_saddle_output_folder, exist_ok=True)

# 讀取 `data.yaml` 獲取類別名稱
yaml_path = "./dataset-yolo-cls-cropped/data.yaml"  # 你的 data.yaml 路徑
with open(yaml_path, "r", encoding="utf-8") as f:
    data_config = yaml.safe_load(f)

# 取得類別名稱對應
class_names = data_config["names"]  # 這是一個 list，對應類別 ID
saddle_class_id = class_names.index("saddle")  # 找到 "saddle" 在 class_names 裡的索引

# 用來存放 `saddle` 和 `not_saddle` 影像的辨識結果
saddle_results = []
not_saddle_results = []

# 進行影像分類
results = final_model.predict(source=input_folder, save=False)  # 不再存 `cls_output`

for result in results:
    # 取得該影像的分類機率
    probs = result.probs  # 取得所有類別的機率

    if probs is not None:  # 確保有機率值
        max_prob_idx = probs.top1  # 取得最高機率的類別索引
        max_prob_value = probs.top1conf  # 取得最高機率值
        image_path = result.path  # 影像路徑
        image_name = os.path.basename(image_path)  # 取得檔案名稱
        
        print(f"📌 影像: {image_name}")
        print(f"🔹 預測類別: {class_names[max_prob_idx]}")
        print(f"🔹 最高機率: {max_prob_value:.4f}\n")

        # ✅ **分類為 `saddle` 且機率 > 0.85**
        if max_prob_idx == saddle_class_id and max_prob_value > 0.85:
            shutil.copy(image_path, os.path.join(saddle_output_folder, image_name))
            saddle_results.append([image_name, max_prob_value])  

        # ✅ **非 "saddle" 或 "saddle" 但機率 ≤ 0.85**
        else:
            shutil.copy(image_path, os.path.join(not_saddle_output_folder, image_name))
            not_saddle_results.append([image_name, max_prob_value, class_names[max_prob_idx]])  

# **儲存 `saddle` 結果為 CSV (只包含機率 > 0.85 的影像)**
saddle_csv_path = os.path.join(saddle_output_folder, "saddle_results.csv")
df_saddle = pd.DataFrame(saddle_results, columns=["Image", "Probability"])
df_saddle.to_csv(saddle_csv_path, index=False)

# **儲存 `not_saddle` 結果為 CSV**
not_saddle_csv_path = os.path.join(not_saddle_output_folder, "not_saddle_results.csv")
df_not_saddle = pd.DataFrame(not_saddle_results, columns=["Image", "Probability", "Predicted_Class"])
df_not_saddle.to_csv(not_saddle_csv_path, index=False)

print(f"✅ `saddle_output` 內含: {os.listdir(saddle_output_folder)}")
print(f"✅ `not_saddle_output` 內含: {os.listdir(not_saddle_output_folder)}")
print(f"📄 `saddle_results.csv` (機率 > 0.85) 已存入: {saddle_csv_path}")
print(f"📄 `not_saddle_results.csv` 已存入: {not_saddle_csv_path}")


image 1/112 /home/user/LAB/Surveying_Camp/SL07_4/mask_0.jpg: 224x224 mudguard 0.38, wheel 0.26, steer 0.20, handle 0.05, background 0.04, 2.2ms
image 2/112 /home/user/LAB/Surveying_Camp/SL07_4/mask_1.jpg: 224x224 wheel 0.98, mudguard 0.02, reflector 0.00, steer 0.00, light 0.00, 1.8ms
image 3/112 /home/user/LAB/Surveying_Camp/SL07_4/mask_10.jpg: 224x224 wheel 0.48, mudguard 0.29, steer 0.10, light 0.03, dress_guard 0.03, 2.2ms
image 4/112 /home/user/LAB/Surveying_Camp/SL07_4/mask_100.jpg: 224x224 saddle 0.99, lock 0.00, pedal 0.00, handle 0.00, reflector 0.00, 1.8ms
image 5/112 /home/user/LAB/Surveying_Camp/SL07_4/mask_101.jpg: 224x224 background 0.53, pedal 0.18, reflector 0.13, handbrake 0.05, handle 0.04, 1.8ms
image 6/112 /home/user/LAB/Surveying_Camp/SL07_4/mask_102.jpg: 224x224 background 0.78, light 0.11, handle 0.02, mudguard 0.02, reflector 0.02, 2.8ms
image 7/112 /home/user/LAB/Surveying_Camp/SL07_4/mask_103.jpg: 224x224 background 0.93, pedal 0.06, handle 0.00, light 0.00, 